In [33]:
%%time
import pandas as pd
from sklearn import cross_validation
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.linear_model import Lasso
#from sklearn.metrics import accuracy_score
#from sklearn.naive_bayes import GaussianNB
#from sklearn.svm import SVC
from sklearn import ensemble
#from sklearn.naive_bayes import BernoulliNB
#from datetime import datetime
#from blaze import Data
#import dask.dataframe as dd
import dask.array as da

CPU times: user 52 µs, sys: 333 µs, total: 385 µs
Wall time: 2.27 ms


In [30]:
%%time
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

CPU times: user 13 µs, sys: 14 µs, total: 27 µs
Wall time: 26.9 µs


In [31]:
%%time
all_train = pd.read_csv('train.csv', dtype=train_dtypes)
#temp_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#all_train = pd.concat(temp_train, ignore_index=True)

CPU times: user 2min 1s, sys: 1min 37s, total: 3min 38s
Wall time: 3min 53s


In [34]:
%%time
#all_train['orig_destination_distance'] = all_train['orig_destination_distance'].fillna(all_train['orig_destination_distance'].median()).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
all_train = all_train.drop(['date_time','orig_destination_distance','srch_ci','srch_co','is_booking', 'cnt'], 1)

CPU times: user 3.07 s, sys: 11.4 s, total: 14.4 s
Wall time: 18.2 s


In [35]:
%%time
#Split into training and test data
split = int(0.70*len(all_train))
#all_train = all_train[split:]
train = all_train[0:split]
test  = all_train[split:]

CPU times: user 616 µs, sys: 797 µs, total: 1.41 ms
Wall time: 1.35 ms


In [36]:
%%time
features_train = train.ix[:,:'hotel_market'] 
labels_train = train.ix[:,'hotel_cluster':]
features_test = test.ix[:,:'hotel_market'] 
labels_test = test.ix[:,'hotel_cluster':]
#features_train = da.from_array(features_train.values, chunks=1000)
#labels_train = da.from_array(labels_train.values, chunks=1000)
#features_train = features_train.values
#labels_train = labels_train.values
#features_test = features_test.values
#labels_test = labels_test.values

CPU times: user 1.86 ms, sys: 2.24 ms, total: 4.1 ms
Wall time: 7.04 ms


In [37]:
%%time
regression = Lasso()
regression.fit(features_train, labels_train.values.ravel())
regression.predict(features_test)
print regression.coef_

[ -2.89779053e-02   0.00000000e+00  -3.17298555e-03   9.34975166e-04
   2.11402910e-07   1.44385683e-07   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -0.00000000e+00
  -1.62837057e-05  -2.07752943e-01  -0.00000000e+00  -1.03258849e-02
   1.95282858e-03]
CPU times: user 18.3 s, sys: 27.5 s, total: 45.8 s
Wall time: 54.8 s


In [24]:
%%time
selector = SelectPercentile(f_classif, percentile=25)
selector.fit(features_train, labels_train.values.ravel())

CPU times: user 22.3 s, sys: 19.6 s, total: 41.9 s
Wall time: 48.2 s


In [23]:
all_train.orig_destination_distance.median()

1140.0

In [21]:
%%time
feature_list = selector.transform(features_train)

CPU times: user 468 ms, sys: 1.21 s, total: 1.68 s
Wall time: 1.7 s


In [22]:
feature_list

array([[   3, 2234,    1,    2,   50],
       [   3, 2234,    1,    2,   50],
       [   3, 2234,    0,    2,   50],
       ..., 
       [   3,  410,    0,    2,   50],
       [   3,  410,    0,    2,   50],
       [   3,   33,    1,    2,   50]])

In [ ]:
%%time
#clf = GaussianNB()
clf = ensemble.AdaBoostClassifier(n_estimators=10)
#clf = ensemble.AdaBoostClassifier(SVC(probability=True, kernel='linear'),n_estimators=10)
#clf = ensemble.GradientBoostingClassifier(SVC(probability=True, kernel='linear'),n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
#clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0, verbose=3)
#clf = ensemble.RandomForestClassifier(n_estimators=10)
#clf - BernoulliNB()
clf = clf.fit(features_train, labels_train.ravel())

In [ ]:
%%time
#pred = clf.predict(features_test)

In [ ]:
#print accuracy_score(pred, labels_test.ravel())

In [ ]:
test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

In [ ]:
%%time
all_test = pd.read_csv('test.csv', dtype=test_dtypes)

In [ ]:
all_test['orig_destination_distance'] = all_test['orig_destination_distance'].fillna(0.0).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
testing_file = all_test.drop(['id', 'date_time','srch_ci','srch_co'], 1)
testing_file = testing_file.values

In [ ]:
%%time
probs = pd.DataFrame(clf.predict_proba(testing_file))

In [ ]:
%%time
probs_series = pd.Series([(i, r.sort_values(ascending=False)[:5].index.values) for i,r in probs.iterrows()])
probs_series = probs_series.values

In [ ]:
%%time
indices = [a for a,b in probs_series]
values = [b for a,b in probs_series]

In [ ]:
%%time
submission = pd.DataFrame()
submission['id'] = indices
submission['hotel_cluster'] = [' '.join(str(x) for x in y) for y in values]

In [ ]:
%%time
submission.to_csv('submission.csv', index=False)